In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# 📌 Cargar datos
df = pd.read_csv(r'C:\Users\alber\OneDrive\Documentos\GitHub\DS_ONLINE_THEBRIDGE_ATC\Masterclass\Kaggle\data\train.csv', index_col="laptop_ID")

# 🔹 Procesamiento de datos
df["Ram"] = df["Ram"].str.replace("GB", "").astype(int)
df["Weight"] = df["Weight"].str.replace("kg", "").astype(float)

# Extraer información detallada de la CPU
def extract_cpu_info(cpu):
    parts = str(cpu).split()
    return f"{parts[1]} {parts[2]}" if len(parts) > 2 and parts[0] == "Intel" else parts[0]
df["Cpu_Brand"] = df["Cpu"].apply(extract_cpu_info)

# Extraer la frecuencia del CPU
def extract_cpu_freq(cpu):
    match = re.search(r'(\d+\.\d+)', str(cpu))
    return float(match.group(1)) if match else np.nan
df["Cpu_Freq_GHz"] = df["Cpu"].apply(extract_cpu_freq)

# Extraer resolución de pantalla y calcular PPI
def extract_resolution(screen_res):
    match = re.search(r'(\d+)x(\d+)', str(screen_res))
    return (int(match.group(1)), int(match.group(2))) if match else (np.nan, np.nan)
df["Resolution_Width"], df["Resolution_Height"] = zip(*df["ScreenResolution"].map(extract_resolution))
df["PPI"] = ((df["Resolution_Width"]**2 + df["Resolution_Height"]**2) ** 0.5) / df["Inches"]

# Identificar Touchscreen
df["Touchscreen"] = df["ScreenResolution"].apply(lambda x: 1 if "Touchscreen" in str(x) else 0)

# Separar memoria en SSD, HDD y Flash
def process_memory(mem):
    ssd, hdd, flash = 0, 0, 0
    parts = str(mem).split()
    for i, part in enumerate(parts):
        if "SSD" in part:
            ssd = int(parts[i-1]) if i > 0 and parts[i-1].isdigit() else 0
        elif "HDD" in part:
            hdd = int(parts[i-1]) if i > 0 and parts[i-1].isdigit() else 0
        elif "Flash" in part:
            flash = int(parts[i-1]) if i > 0 and parts[i-1].isdigit() else 0
    return pd.Series([ssd, hdd, flash])
df[['SSD', 'HDD', 'Flash']] = df['Memory'].apply(process_memory)

# Extraer GPU Brand
def extract_gpu_brand(gpu):
    brands = ["Nvidia", "AMD", "Intel"]
    return next((brand for brand in brands if brand in str(gpu)), "Other")
df["Gpu_Brand"] = df["Gpu"].apply(extract_gpu_brand)

# Codificar sistema operativo (Windows o no)
df['Windows'] = df['OpSys'].apply(lambda x: 1 if 'Windows' in str(x) else 0)

# 📌 Seleccionar características
X = df.drop(columns=["Price_in_euros", "Product", "Cpu", "OpSys", "Memory", "ScreenResolution", "Gpu"])
y = df["Price_in_euros"]

# 📌 Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 📌 Identificar variables categóricas y numéricas
categorical_features = ["Company", "TypeName", "Cpu_Brand", "Gpu_Brand"]
numeric_features = ["Inches", "Ram", "Weight", "Resolution_Width", "Resolution_Height", "PPI",
                    "SSD", "HDD", "Flash", "Windows", "Cpu_Freq_GHz", "Touchscreen"]

# 📌 Preprocesador con imputación para valores NaN
numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

# 📌 Optimización de hiperparámetros
param_dist = {
    "regressor__n_estimators": [100, 300, 500, 800],
    "regressor__learning_rate": [0.01, 0.05, 0.1, 0.2],
    "regressor__max_depth": [3, 5, 7, 10],
    "regressor__min_samples_split": [2, 4, 6, 10],
    "regressor__min_samples_leaf": [1, 2, 5]
}

model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", GradientBoostingRegressor(random_state=42))
])

search = RandomizedSearchCV(model, param_dist, cv=5, scoring='neg_root_mean_squared_error', n_iter=20, n_jobs=-1, random_state=42)
search.fit(X_train, y_train)

# 📌 Evaluar modelo
best_model = search.best_estimator_
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Optimized RMSE: {rmse}")

# 📌 Cargar datos de prueba
X_pred = pd.read_csv(r'C:\Users\alber\OneDrive\Documentos\GitHub\DS_ONLINE_THEBRIDGE_ATC\Masterclass\Kaggle\data\test.csv', index_col="laptop_ID")

# 📌 Aplicar el mismo procesamiento
X_pred["Ram"] = X_pred["Ram"].str.replace("GB", "").astype(int)
X_pred["Weight"] = X_pred["Weight"].str.replace("kg", "").astype(float)
X_pred["Cpu_Brand"] = X_pred["Cpu"].apply(extract_cpu_info)
X_pred["Cpu_Freq_GHz"] = X_pred["Cpu"].apply(extract_cpu_freq)
X_pred[['SSD', 'HDD', 'Flash']] = X_pred['Memory'].apply(process_memory)
X_pred["Gpu_Brand"] = X_pred["Gpu"].apply(extract_gpu_brand)
X_pred["Windows"] = X_pred["OpSys"].apply(lambda x: 1 if 'Windows' in str(x) else 0)
X_pred["Touchscreen"] = X_pred["ScreenResolution"].apply(lambda x: 1 if "Touchscreen" in str(x) else 0)
X_pred["Resolution_Width"], X_pred["Resolution_Height"] = zip(*X_pred["ScreenResolution"].map(extract_resolution))
X_pred["PPI"] = ((X_pred["Resolution_Width"]**2 + X_pred["Resolution_Height"]**2) ** 0.5) / X_pred["Inches"]

# 📌 Eliminar columnas innecesarias (Solo ahora después de usarlas)
X_pred = X_pred.drop(columns=["Product", "Cpu", "OpSys", "Memory", "ScreenResolution", "Gpu"])

# 📌 Hacer predicciones
predictions = best_model.predict(X_pred)


Optimized RMSE: 373.3784445712431
